In [6]:
import cv2
import numpy as np

In [31]:
#apply gaussian blur
def gaussianblur(image, kernel_size=3):
    # Generate Gaussian kernel
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32)
    kernel /= kernel_size ** 2
    
    # Convolve image with Gaussian kernel
    blurred_image = cv2.filter2D(image, -1, kernel)
    
    return blurred_image



In [32]:
def gradient_intensity(image):
    # Sobel kernels
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32)
    
    # Compute gradients
    gradient_x = cv2.filter2D(image, -1, kernel_x)
    gradient_y = cv2.filter2D(image, -1, kernel_y)
    
    # Compute gradient magnitude
    gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)
    
    # Normalize gradient magnitude to [0, 255]
    gradient_magnitude = (gradient_magnitude / np.max(gradient_magnitude)) * 255
    
    return gradient_magnitude



In [33]:
#gradient magnitude thresholding or lower bound cut-off suppression
def threshold(image, threshold):
    thresholded_image = np.where(image > threshold, image, 0)
    return thresholded_image



In [34]:
#double thresholding
def double_threshold(image, low_threshold, high_threshold):
    strong_edges = np.where(image > high_threshold, image, 0)
    weak_edges = np.where((image <= high_threshold) & (image > low_threshold), image, 0)
    return strong_edges, weak_edges



In [35]:
#edge tracking by hysteresis
def hysteresis(image, weak_edges, strong_edges):
    height, width = image.shape
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            if weak_edges[i, j] > 0:
                if strong_edges[i - 1, j - 1] or strong_edges[i - 1, j] or strong_edges[i - 1, j + 1] or \
                   strong_edges[i, j - 1] or strong_edges[i, j + 1] or \
                   strong_edges[i + 1, j - 1] or strong_edges[i + 1, j] or strong_edges[i + 1, j + 1]:
                    image[i, j] = 255
                else:
                    image[i, j] = 0
    return image




In [36]:

#Canny Edge Detection
def canny_edge_detection(image, kernel_size, low_threshold, high_threshold):
    # Apply Gaussian blur
    blurred_image = gaussianblur(image, kernel_size)
    
    # Compute gradient magnitude and direction
    gradient_magnitude = gradient_intensity(blurred_image)
    
    # Apply double threshold
    strong_edges, weak_edges = double_threshold(gradient_magnitude, low_threshold, high_threshold)
    
    # Edge tracking by hysteresis
    edge_tracked_image = hysteresis(gradient_magnitude, weak_edges, strong_edges)
    
    return edge_tracked_image




In [51]:
image = cv2.imread("cat.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
canny_image = canny_edge_detection(gray_image, kernel_size=6, low_threshold=60, high_threshold=100)

cv2.imshow("Canny Image", canny_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()
